In [7]:
import pandas as pd

In [8]:
market_data_path = "./data/test_prices_day_0.csv"
trade_history_path = "./data/test_trades_day_0.csv"

In [9]:
market_data = pd.read_csv(market_data_path, delimiter=";")

In [10]:
market_data.head()

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,0,0,AMETHYSTS,9998,1,9995.0,30.0,NaN,NaN,10005,30,NaN,NaN,NaN,NaN,10001.5,0.0
1,0,0,STARFRUIT,5036,30,NaN,NaN,NaN,NaN,5043,30,NaN,NaN,NaN,NaN,5039.5,0.0
2,0,100,AMETHYSTS,9996,1,9995.0,30.0,NaN,NaN,10002,6,10004.0,1.0,10005.0,30.0,9999.0,0.0
3,0,100,STARFRUIT,5041,4,5037.0,1.0,5036.0,30.0,5043,31,NaN,NaN,NaN,NaN,5042.0,0.0
4,0,200,AMETHYSTS,9996,1,9995.0,21.0,NaN,NaN,10004,1,10005.0,21.0,NaN,NaN,10000.0,0.0


In [11]:
trade_history = pd.read_csv(trade_history_path, delimiter=";")

In [12]:
trade_history.head()

,timestamp,buyer,seller,symbol,currency,price,quantity
0,100,NaN,NaN,STARFRUIT,SEASHELLS,5041.0,4
1,100,NaN,NaN,AMETHYSTS,SEASHELLS,10002.0,1
2,300,NaN,NaN,AMETHYSTS,SEASHELLS,9998.0,2
3,300,NaN,NaN,AMETHYSTS,SEASHELLS,9998.0,1
4,300,NaN,NaN,STARFRUIT,SEASHELLS,5037.0,6


In [13]:
from run import Trader

In [105]:
from datamodel import Listing, Trade, OrderDepth, TradingState, Observation
from collections import defaultdict

class Backtest:
    """
        Simulate IMC exchange locally
    """
    def __init__(self, trader, listings, position_limits, market_data, trade_history, output_log):
        self.trader = trader
        self.listings = listings
        self.position_limits = position_limits
        self.market_data = market_data.sort_values(by='timestamp')
        # TODO (kyraz): I don't think we need to sort by symbol
        self.trade_history = trade_history.sort_values(by=['timestamp', 'symbol'])
        self.output_log = output_log

    def run(self):
        """
        Logic:
            - Iterate through market data timestamps, starting from market open t_0
            - For timestamp t_i, replay all market trades in trade history between t_{i-1} and t_i
            - For timestamp t_i, compute the markout pnl for each product at the current position
        """

        market_data_gp_ts = self.market_data.groupby('timestamp')
        trade_history_gp_ts = self.trade_history.groupby('timestamp')

        # Group trades by timestamp
        trades_by_timestamp = defaultdict()
        for timestamp, group in trade_history_gp_ts:
            trades = [
                Trade(
                    symbol=trade['symbol'], 
                    price=trade['price'],
                    quantity=trade['quantity'],
                    buyer=trade['buyer'],
                    seller=trade['seller'],
                    timestamp=trade['timestamp']
                ) for _, trade in group.iterrows()]
            trades_by_timestamp[timestamp] = trades

        for timestamp, group in market_data_gp_ts:
            market_trades = trades_by_timestamp[timestamp]


            # TODO: ADD, DELETE, EXECUTE orders
            # TODO: Construct order book
            # TODO: Construct trading state (positions)
            # TODO: Compute PnL
        pass

    def _add_order(self):
        pass

    def _delete_order(self):
        pass

    def _execute_order(self, trades_to_execute):
        pass

    def calc_pnl(self):
        pass

In [87]:
#Debugging
market_data_gp_ts = market_data.groupby('timestamp')
trade_history_gp_ts = trade_history.groupby('timestamp')

# Group trades by timestamp
trades_by_timestamp = {}
for timestamp, group in trade_history_gp_ts:
    trades = [
        Trade(
            symbol=trade['symbol'], 
            price=trade['price'],
            quantity=trade['quantity'],
            buyer=trade['buyer'] if not np.isnan(trade['buyer']) else "",
            seller=trade['seller'] if not np.isnan(trade['buyer']) else "",
            timestamp=trade['timestamp']
        ) for _, trade in group.iterrows()]
    trades_by_timestamp[timestamp] = trades

In [107]:
current_position = 0
observations = None

In [111]:
LEVELS = 3
trader_data = ""
symbols = [listing.symbol for listing in listings]
# Own trades since last timestamp
own_trades = {listing.symbol: [] for listing in listings}
# Market trades since last timestamp
market_trades = {listing.symbol: [] for listing in listings}
for timestamp, group in market_data_gp_ts:
    order_depths = {}
    
    # Construct order book
    for _, row in group.iterrows():
        symbol = row['product']
        order_depth = OrderDepth()
        for i in range(1, LEVELS+1):
            # Bid
            bid_price = row[f'bid_price_{i}']
            bid_volume = row[f'bid_volume_{i}']
            if not np.isnan(bid_price) and not np.isnan(bid_volume):
                order_depth.buy_orders[int(bid_price)] = int(bid_volume)
                
            # Ask
            ask_price = row[f'ask_price_{i}']
            ask_volume = row[f'ask_volume_{i}']
            if not np.isnan(ask_price) and not np.isnan(ask_volume):
                order_depth.sell_orders[int(ask_price)] = int(ask_volume)

            order_depths[symbol] = order_depth

    # Assemble trading state
    trading_state = TradingState(trader_data, timestamp, listings, order_depths, own_trades, market_trades, current_position, observations)
    orders, conversions, trader_data = trader.run(trading_state)

    # Update market trades status
    market_trades = {listing.symbol: [] for listing in listings}
    if timestamp in trades_by_timestamp.keys():
        for trade in trades_by_timestamp[timestamp]:
            market_trades[trade.symbol].append(trade)

    # TODO: Execute own orders

traderData: 
Observations: None
Acceptable price : 10
Buy Order depth : 2, Sell order depth : 1
SELL 1x 9998
Acceptable price : 10
Buy Order depth : 1, Sell order depth : 1
SELL 30x 5036
traderData: SAMPLE
Observations: None
Acceptable price : 10
Buy Order depth : 2, Sell order depth : 3
SELL 1x 9996
Acceptable price : 10
Buy Order depth : 3, Sell order depth : 1
SELL 4x 5041
traderData: SAMPLE
Observations: None
Acceptable price : 10
Buy Order depth : 2, Sell order depth : 2
SELL 1x 9996
Acceptable price : 10
Buy Order depth : 2, Sell order depth : 2
SELL 1x 5037
traderData: SAMPLE
Observations: None
Acceptable price : 10
Buy Order depth : 1, Sell order depth : 1
SELL 25x 5037
Acceptable price : 10
Buy Order depth : 2, Sell order depth : 3
SELL 2x 9996
traderData: SAMPLE
Observations: None
Acceptable price : 10
Buy Order depth : 3, Sell order depth : 2
SELL 5x 9998
Acceptable price : 10
Buy Order depth : 1, Sell order depth : 2
SELL 25x 5037
traderData: SAMPLE
Observations: None
Accep

In [112]:
orders

{'STARFRUIT': [(STARFRUIT, 5048, -2)], 'AMETHYSTS': [(AMETHYSTS, 9996, -2)]}

In [116]:
order_depths['STARFRUIT'].buy_orders

{5048: 2, 5047: 21}

In [ ]:
def execute_buy_order(order, order_depths_symbol, positions_symbol, positions_limit_symbol):
    order_symbol, order_price, order_volume = order
    order_abs_volume = abs(order_volume)
    # Only care about the ask side
    for sell_price, sell_abs_volume in list(order_depths_symbol.sell_orders.items()):
        if order_abs_volume <= 0 or sell_price > order_price:
            break
        executed_volume = min(min(sell_abs_volume, order_abs_volume), positions_limit_symbol - abs(positions_symbol))
        if executed_volume > 0:
            remain_volume = order_abs_volume - executed_volume
            order_abs_volume = remain_volume

        # Update order book

In [ ]:
def execute_order(orders, order_depths):
    for symbol in symbols:
        orders_for_symbol = orders[symbol]
        order_depths_symbol = order_depths[symbol]
        for order in orders_for_symbol:
            order_symbol, order_price, order_volume = order
            if order_volume > 0:
                # Execute buy order
                execute_buy_order(order, order_depths_symbol)
            else:
                # Execute sell order
                execute_sell_order(order, order_depths_symbol)

In [101]:
order_depths['AMETHYSTS'].buy_orders

{9996: 2, 9995: 21}

In [82]:
TradingState(
    traderData = str,
    timestamp = Time,
    listings = listings,
    order_depths = order_depths,
    own_trades = Dict[Symbol, List[Trade]],
    market_trades: Dict[Symbol, List[Trade]],
    position: Dict[Product, Position],
    observations: Observation)

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
2,0,100,AMETHYSTS,9996,1,9995.0,30.0,NaN,NaN,10002,6,10004.0,1.0,10005.0,30.0,9999.0,0.0
3,0,100,STARFRUIT,5041,4,5037.0,1.0,5036.0,30.0,5043,31,NaN,NaN,NaN,NaN,5042.0,0.0


In [36]:
listings = [
    Listing(symbol='AMETHYSTS', product='AMETHYSTS', denomination='SEASHELLS'),
    Listing(symbol='STARFRUIT', product='STARFRUIT', denomination='SEASHELLS'),
]

position_limit = {
    'AMETHYSTS': 20,
    'STARFRUIT': 20
}

In [41]:
output_log = "backtest.log"
trader = Trader()
backtest = Backtest(trader, listings, position_limit, market_data, trade_history, output_log)
backtest.run()

KeyError: 0